<a href="https://colab.research.google.com/github/AcererakTheMighty/FirstMLProject/blob/main/FirstProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.6/446.6 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 35.9 M

In [2]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [3]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [14]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [15]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION




Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=dee85e18-4dec-437a-9b07-6c8e48f2e60d
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


70 relevant feature(s) found with the search keys: ['date']


f_autofe_mul_34d11bc4,0.0237,100.0000,"8.3373, -12.5377, -6.659",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_50507976,0.0208,100.0000,"-0.0505, 0.0513, 0.0578",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_135dc79d,0.0179,100.0000,"0.6775, 0.8103, 0.7992",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_0cc09d2d,0.0136,100.0000,"-98.149, 57.4259, 79.6239",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_478be4f3,0.0124,100.0000,"0.7927, -0.7418, -0.7719",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_89c56a5f,0.0111,100.0000,"-0.3941, 0.5888, 1.0209",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_cf5dd7be,0.0091,100.0000,"-0.9473, 0.9904, -0.7728",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_events_date_year_cos1_9014a856,0.0086,100.0000,"-0.7988, 0.8703, 0.9357",Upgini,Calendar data,Daily
f_autofe_mul_402f5e6a,0.0075,100.0000,"1.0707, 0.0915, 0.5096",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_283a6178,0.0070,100.0000,"-0.0022, -0.0112, 0.0063",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_3e22df83,0.0065,100.0000,"0.0226, 0.0043, 0.0096",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,"AutoFE: features from Calendar data,Markets data",0.1493,39
Upgini,AutoFE: features from Markets data,0.0292,18
Upgini,Calendar data,0.0089,2
Upgini,AutoFE: features from Calendar data,0.0060,9
Upgini,Markets data,0.0003,1
Upgini,AutoFE: feature from Markets data,0.0001,1


"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_50507976,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,/
Markets data,f_autofe_div_135dc79d,f_financial_date_gold_7d_to_1y_ae310379,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_mul_0cc09d2d,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_mul_478be4f3,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_mul_402f5e6a,f_events_date_year_cos1_9014a856,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,*
"Calendar data,Markets data",f_autofe_div_283a6178,f_events_date_week_cos2_b0a07cfc,f_financial_date_usd_7419609a,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_mul_08e89348,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,*



Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,301.4672,201.8016,99.6656
Eval 1,3787,59.2424,486.0393,345.4817,140.5576


In [16]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.252850,0.175309,0.077541
1,Eval 1,3787,59.2424,0.264187,0.178654,0.085533


In [17]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

[============================================================] 100% Finished

You are trying to launch enrichment for 15213 rows, which will exceed the rest limit 10000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=0a92d8b4-9091-4f94-bfb3-56d3012cfebb
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")



In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")